In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="p")
index = pc.Index("test")

In [ ]:
texts = [
    "Sudhanshu Kumar was born in Jamshedpur, Jharkhand, India, to a modest family.",
    "His early life was marked by financial hardship and limited resources.",
    "He believed education could transform his life and pursued it relentlessly.",
    "Sudhanshu earned a Computer Science and Engineering degree.",
    "He worked at Wipro, Deloitte, Verizon Labs, and Ernst & Young.",
    "He gained expertise in SAP WebDynpro, Fiori UI5 HANA, Java, Big Data, and Data Analytics.",
    "Sudhanshu remained committed to making education accessible for everyone.",
    "His teaching empowered countless students to build their careers.",
    "Many students credit him with changing their lives through affordable learning.",
    "His journey is a testament to triumph over adversity and the power of knowledge."
]


import requests
import numpy as np

def generate_embeddings(text):
    url = "https://api.euron.one/api/v1/euri/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer "
    }
    payload = {
        "input": text,
        "model": "text-embedding-3-small"
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    
    embedding = np.array(data['data'][0]['embedding'])
    
    return embedding

embeddings = [generate_embeddings(text).tolist() for text in texts]

In [8]:
records = [(str(i), embeddings[i], {"text": texts[i]}) for i in range(len(texts))]


In [9]:
index.upsert(vectors=records)

{'upserted_count': 10}

In [10]:
query = "Sudhanshu worked at Deloitte!"
query_embed = generate_embeddings(query).tolist()

In [11]:
index.query(vector=query_embed, top_k=2, include_metadata=True)

{'matches': [{'id': '4',
              'metadata': {'text': 'He worked at Wipro, Deloitte, Verizon '
                                   'Labs, and Ernst & Young.'},
              'score': 0.52022171,
              'values': []},
             {'id': '3',
              'metadata': {'text': 'Sudhanshu earned a Computer Science and '
                                   'Engineering degree.'},
              'score': 0.598967612,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 1}}